# Criação de um template do excel para manipulação de dados
Este template será substituído posteriormente pela tabela que de fato corresponde aos cursos e trilhas do datacamp

In [47]:
import pandas as pd

membros_feadev = pd.DataFrame({
        'id_membro': [0, 1, 2],
        'nome': ['admin', 'Tiago Toledo', 'Rogério Ceni'],
        'email': ['email-falso-admin-dev@gmail.com', 'ttduarte@usp.br', 'rogerio-ceni@gmail.com'],
        'conta_github': ['admin-feadev-github', 'Tiago745', 'Rogerio-Ceni-Github'],
        'conta_datacamp': ['', '', ''], #provavelmente será uma url para cursos finalizados, ainda precisa ser revisado
        'xp_datacamp': ['', '', '']
})

trilhas = pd.DataFrame({
        'id_trilha': [0, 1, 2], #chave/id/pk
        'nome_trilha': ['Associate Data Scientist in Python', 'Associate Data Engineer in SQL',  'Capacitação 2025 - Básico'],
        'url': ["https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python","https://app.datacamp.com/learn/career-tracks/associate-data-engineer-in-sql", ''],
        'tipo_trilha': [0,0,1] # 0-> trilha do datacamp || 1-> trilha personalizada (default deve ser 0, 1 somente quando for personalizado ou criado in-app)
        })

cursos = pd.DataFrame({
    'id_curso': [0, 1, 2],
    'nome_curso': ['Introduction to Python', 'Intermediate Python', 'SQL'],
    'duracao' : [4, 4, 4]
})


#DataFrames Associativos
trilhas_tem_cursos = pd.DataFrame({
    'id_trilha' : [0, 0, 1, 2, 2, 2],
    'id_curso' : [0, 1, 2, 0, 1, 2],
    'ordem_curso' : [0, 1, 0, 1, 2, 0], #ordem em que os cursos devem ser assistidos dentro de cada trilha
    'data_final_para_assistir': ['', '', '','20/06/2023', '20/06/2023', '23/06/2023'], #cursos de trilhas do datacamp nao tem data-final, apenas trilhas personalizadas
    'obrigatoriedade_curso': [1, 1, 1, 1, 1, 1] #o curso precisa ser obrigatoriamente assistido ou é opcional, 1->Obrigatório 0->Não obrigatório/opcional
})

membro_feadev_faz_trilhas = pd.DataFrame({
    'id_membro' : [1],
    'id_trilha' : [2],
    'data_inicio' : ['15/06/2025'],
    'data_fim' : ['20/06/2025'],
    'finalizado' : [True]
})

membro_feadev_faz_cursos = pd.DataFrame({
    'id_membro' : [1, 1, 1],
    'id_curso' : [0,1,2],
    'data_inicio' : ['15/06/2025', '17/06/2025', '20/06/2025'],
    'data_fim' : ['15/06/2025', '21/06/2025', '21/06/2025'],
    'finalizado' : [True, True, True]
})

#Uniao de trilhas com cursos
resultado = trilhas.merge(trilhas_tem_cursos, on='id_trilha') \
                   .merge(cursos, on='id_curso') \
                   #[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso']]  # remove colunas extras

resultado_ordenado = resultado.sort_values(by=['id_trilha', 'ordem_curso']) #ordena por trilha e depois por ordem que deve assistir cada curso, facilita a leitura

print(resultado_ordenado[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso', 'ordem_curso']])


# Caminho do arquivo Excel que será salvo
arquivo_excel = 'dados_trilhas.xlsx'

# Usando o ExcelWriter para salvar cada DataFrame em uma aba diferente
with pd.ExcelWriter('arquivo.xlsx', engine='openpyxl') as writer:
    trilhas.to_excel(writer, sheet_name='Trilhas', index=False)
    cursos.to_excel(writer, sheet_name='Cursos', index=False)
    trilhas_tem_cursos.to_excel(writer, sheet_name='Trilhas_tem_Cursos', index=False)
    membros_feadev.to_excel(writer, sheet_name='Membros_feadev', index=False)
    membro_feadev_faz_trilhas.to_excel(writer, sheet_name='membro_feadev_faz_trilhas', index=False)
    membro_feadev_faz_cursos.to_excel(writer, sheet_name='membro_feadev_faz_cursos', index=False)

print("✅ Arquivo Excel salvo com sucesso!")


   id_trilha                         nome_trilha  id_curso  \
0          0  Associate Data Scientist in Python         0   
1          0  Associate Data Scientist in Python         1   
2          1      Associate Data Engineer in SQL         2   
5          2           Capacitação 2025 - Básico         2   
3          2           Capacitação 2025 - Básico         0   
4          2           Capacitação 2025 - Básico         1   

               nome_curso  ordem_curso  
0  Introduction to Python            0  
1     Intermediate Python            1  
2                     SQL            0  
5                     SQL            0  
3  Introduction to Python            1  
4     Intermediate Python            2  
✅ Arquivo Excel salvo com sucesso!


# Atualizar cursos e trilhas do datacamp
Por enquanto só atualiza a tabela de trilhas (falta cursos, e trilhas_tem_cursos)

In [ ]:
import pandas as pd
import os

#Recupera o nome de cada arquivo dentro da subpasta trilhas_atualizadas
LISTA_DE_TRILHAS_PARA_LER = [f for f in os.listdir("trilhas_atualizadas") if os.path.isfile(os.path.join("trilhas_atualizadas", f))]



def atualizar_trilhas_com_ids(trilhas_atualizadas_datacamp, maedev_df, col_nome_trilha_atualizada="Trilha", col_df_maedev=['id_trilha', 'nome_trilha', 'url', 'tipo_trilha']):
    """
    Adiciona novas trilhas únicas de `trilhas_atualizadas_datacamp` em `maedev_df`, com id incremental.
    
    Parâmetros:
        trilhas_atualizadas_datacamp (pd.DataFrame): DataFrame com os cursos e suas trilhas.
        maedev_df (pd.DataFrame): DataFrame com trilhas já cadastradas.
        col_nome_trilha_atualizada (str): Nome da coluna no trilhas_atualizadas_datacamp que contém o nome das trilhas.
        col_df_maedev (list): Lista com os nomes das colunas na ordem: [id, nome, url, tipo].

    Retorna:
        pd.DataFrame: O DataFrame `maedev_df` atualizado com novas trilhas e IDs únicos.
    """
    url_padrao_trilha = 'https://app.datacamp.com/learn/career-tracks/'
    
    # Padroniza os nomes das trilhas
    trilhas_atualizadas_datacamp[col_nome_trilha_atualizada] = trilhas_atualizadas_datacamp[col_nome_trilha_atualizada].astype(str).str.strip()
    maedev_df[col_df_maedev[1]] = maedev_df[col_df_maedev[1]].astype(str).str.strip()

    # ID inicial baseado no maior ID atual
    ultimo_id = maedev_df[col_df_maedev[0]].max() if not maedev_df.empty else -1
    novas_linhas = []

    for trilha in trilhas_atualizadas_datacamp[col_nome_trilha_atualizada].unique():
        if trilha not in maedev_df[col_df_maedev[1]].values:
            ultimo_id += 1
            novas_linhas.append({
                col_df_maedev[0]: ultimo_id,
                col_df_maedev[1]: trilha,
                #col_df_maedev[2]: (url_padrao_trilha + trilha.lower().replace("  ", " ")).replace(' ', '-'),
                col_df_maedev[2]: url_padrao_trilha + "-".join(trilha.lower().split()), #algumas urls tem erros de digitacao com espaços duplos
                col_df_maedev[3]: 0  # 0 = oficial, 1 = personalizada
            })
            print(f"✅ Nova trilha adicionada: {trilha} (id = {ultimo_id})")

    # Adiciona as novas trilhas, se houver
    if novas_linhas:
        df_novos = pd.DataFrame(novas_linhas)
        maedev_df = pd.concat([maedev_df, df_novos], ignore_index=True)
        print("🎯 Atualização concluída com sucesso.")
    else:
        print("ℹ️ Nenhuma nova trilha encontrada.")

    return maedev_df

# Lê o arquivo com o Banco de Dados do aplicativo MAE dev
df_mae_dev = pd.read_excel("arquivo.xlsx", sheet_name='Trilhas')

#
for trilha in LISTA_DE_TRILHAS_PARA_LER:
    cursos_atualizados = pd.read_excel('./trilhas_atualizadas/'+trilha)
    df_mae_dev = atualizar_trilhas_com_ids(cursos_atualizados, df_mae_dev)

# (Opcional) salvar o resultado
df_mae_dev.to_excel("trilhas_atualizadas.xlsx", index=False)

✅ Nova trilha adicionada: Associate Data Scientist  in Python (id = 3)
🎯 Atualização concluída com sucesso.
✅ Nova trilha adicionada: Excel Fundamentals (id = 4)
🎯 Atualização concluída com sucesso.
✅ Nova trilha adicionada: Git Fundamentals (id = 5)
🎯 Atualização concluída com sucesso.
✅ Nova trilha adicionada: SQL Fundamentals (id = 6)
🎯 Atualização concluída com sucesso.


# Placeholder de Webscrapping (ainda não implementado)

In [2]:
from cloudscraper import create_scraper # biblioteca para contornar proteção de sites que usam Cloudflare
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright # biblioteca para automação de navegadores

import json 
import random
import pickle # biblioteca para salvar e carregar objetos Python em arquivos
import asyncio
import requests

scraper = create_scraper()

urls_courses = ["https://app.datacamp.com/learn/career-tracks/associate-data-engineer-in-sql", "https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python"]
urls_skills = ["https://app.datacamp.com/certification/get-started/data-engineer-associate/overview", "https://app.datacamp.com/certification/get-started/associate-data-scientist/overview"]


response = requests.get(urls_courses[0])
print(response.status_code) # erro 403: Forbidden, indica que a requisição foi negada pelo servidor

response = scraper.get(urls_courses[0]) # usando o scraper para contornar a proteção do site
print(response.status_code) # agora deve retornar 200, indicando sucesso

# soup = BeautifulSoup(response.text, "html.parser") # analisa o HTML da página
# print(soup.prettify()) # imprime o HTML formatado 

semaphore = asyncio.Semaphore(5) # Limita o número de páginas abertas simultaneamente para 5

# extrair o conteudo das habilidades de cada trilha
async def extract_skills(context, url):
    async with semaphore:
        page = await context.new_page()
        try: # tenta acessar a página
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4))
            
            if "just a moment" not in (await page.content()).lower(): # Verifica se a página não foi redirecionada para uma página de proteção
                soup = BeautifulSoup(await page.content(), "html.parser") # analisa o HTML da página
                
                skills = list(set([skills.get_text(strip=True) # extrai as habilidades (sem duplicatas)
                        for skills in soup.select("section ul span")])) # (section ul span) é o seletor CSS que localiza as habilidades
                
                track_element = soup.find("h1") # localiza o elemento que contém o nome da trilha
                track = track_element.get_text(strip=True) if track_element else "N/A" # extrai o texto do elemento, se encontrado

                data = []
                for skill in skills:
                    data.append({"track": track, "skill": skill})# cria um dicionário com a trilha e a habilidade
                return data
            
            else:
                print("❌ Falha: Cookies expirados ou inválidos. Repita o login manual.")
                return None
        except Exception as e: # se ocorrer um erro ao acessar a página
            print(f"❌ Erro ao extrair habilidades da página {url}: {e}")
            return None
        finally: # garante que a página será fechada
            await page.close()
            print(f"✅ Página {url} fechada.")

# extrair o conteudo dos cursos de cada trilha
async def extract_courses(context, url):
    async with semaphore:
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4)) # espera um tempo aleatório entre 2 e 4 segundos para evitar bloqueios por parte do site
            
            if "just a moment" not in (await page.content()).lower():
                soup = BeautifulSoup(await page.content(), "html.parser")

                track_element = soup.find("h1")
                track = track_element.get_text(strip=True) if track_element else "N/A"

                projects = {optional.find_parent('div') for optional in soup.select(".mfe-app-learn-hub-1moscjt")} # projetos opcionais do curso o ".mfe-app-learn-hub-1moscjt" é o seletor CSS que localiza os projetos

                courses = list(set([course.get_text(strip=True) for course in soup.select("h3.mfe-app-learn-hub-1yqo1j7") # h3.mfe-app-learn-hub-1yqo1j7 é o seletor CSS que localiza os cursos
                                if course.find_parent('div') not in projects])) # extrai apenas os cursos
                await page.close()
                duration = {}
                try: # tenta acessar a página de cada curso para extrair a duração
                    for course in courses:
                        course_page = course.replace(" ", "-").lower() # formata o nome do curso para criar a URL
                        tmp_url = f"https://app.datacamp.com/learn/courses/{course_page}" 
                        page = await context.new_page() 

                        await page.goto(tmp_url, timeout=60000)
                        await asyncio.sleep(random.uniform(2, 3))

                        soup = BeautifulSoup(await page.content(), "html.parser")

                        duration_element = soup.select_one(".mfe-app-learn-hub-hdd90k")# seletor CSS que localiza a duração do curso
                        duration[course] = duration_element.get_text(strip=True) if duration_element else "N/A"

                        await page.close()

                    print(f"✅ Duração de curso extraídos com sucesso da página {url}.")
                except Exception as e:
                    print(f"❌ Erro ao acessar a página do curso: {e}")
                    return None
                data = []
                for course in courses:
                    data.append({"track": track, "course": course, "duration": duration[course]})
                return data
            else:
                print("❌ Falha: Cookies expirados ou inválidos. Repita o login manual.")
                return None
        except Exception as e:
            print(f"❌ Erro ao extrair curso da página {url}: {e}")
            return None
            



403
403


# Criando cookies 

In [2]:
from playwright.async_api import async_playwright
import pickle
import asyncio
import os
import nest_asyncio

nest_asyncio.apply()  # permite rodar async no Jupyter/Colab

url = "https://app.datacamp.com"
cookies_path = "./cookies/datacamp_cookies.pkl"

async def save_cookies_manually():
    os.makedirs(os.path.dirname(cookies_path), exist_ok=True)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(url, timeout=60000)
        await context.clear_cookies()

        input("⚠️ Faça login no navegador que abriu. Depois pressione Enter aqui...")

        cookies = await context.cookies()
        with open(cookies_path, "wb") as f:
            pickle.dump(cookies, f)

        print(f"✅ Cookies salvos com sucesso em: {cookies_path}")

# Em Jupyter ou Colab:
await save_cookies_manually()

✅ Cookies salvos com sucesso em: ./cookies/datacamp_cookies.pkl


# Gerando arquivo com os cursos do datacamp
### Gera um xlsx com a ata de cursos de cada trilha dentro da lista LISTA_DE_TRILHAS_PARA_ATUALIZAR

In [74]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pickle
import os
import asyncio
import nest_asyncio
import random
import pandas as pd

'''
Mude as urls para atualizar trilhas diferentes do datacamp, 
Para ser mais rápido, atualize uma por vez e eviter atualizar coisas desnecessárias
Funciona tanto para skill tracks quanto carrer tracks
'''
LISTA_DE_TRILHAS_PARA_ATUALIZAR = [
    #trilhas da capacitação básica 2025
    #"https://app.datacamp.com/learn/career-tracks/data-scientist-with-python",
    #'https://app.datacamp.com/learn/skill-tracks/sql-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/git-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/excel-fundamentals',

    #trilhas da capacitação avançada 2025
    #'https://app.datacamp.com/learn/skill-tracks/time-series-with-python', #curso de Séries temporais
    #'https://app.datacamp.com/learn/skill-tracks/applied-finance-in-python', #curso de Introdução ao Gerenciamento de Risco de Portfólio em Python e curso de Garch
    #'https://app.datacamp.com/learn/skill-tracks/finance-fundamentals-in-python', #curso de Introdução à Análise de Portfólio em python
    'https://app.datacamp.com/learn/career-tracks/machine-learning-scientist-with-python' #contem boa parte dos cursos de machine learning usados
     
]
'''
CURSOS SEM TRILHAS (PRECISAM SER ADICIONADOS MANUALMENTE)
https://app.datacamp.com/learn/courses/introduction-to-optimization-in-python
https://app.datacamp.com/learn/courses/introduction-to-deep-learning-in-python
'''

nest_asyncio.apply()

cookies_path = "./cookies/datacamp_cookies.pkl"
semaphore = asyncio.Semaphore(3)

# Cria o contexto autenticado com os cookies
async def create_authenticated_context(playwright):
    if not os.path.exists(cookies_path):
        print("❌ Arquivo de cookies não encontrado. Faça o login e salve os cookies primeiro.")
        return None, None

    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()

    with open(cookies_path, "rb") as f:
        cookies = pickle.load(f)
    await context.add_cookies(cookies)

    return browser, context

# Extrai os cursos e durações de uma trilha
async def extract_courses(context, url):
    async with semaphore:
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4))

            if "just a moment" not in (await page.content()).lower():
                soup = BeautifulSoup(await page.content(), "html.parser")

                track_element = soup.find("h1")
                track = track_element.get_text(strip=True) if track_element else "N/A"

                projects = {
                    optional.find_parent("div")
                    for optional in soup.select(".mfe-app-learn-hub-1moscjt")
                }

                courses = list([
                    course.get_text(strip=True)
                    for course in soup.select("h3.mfe-app-learn-hub-1yqo1j7")
                    if course.find_parent("div") not in projects
                ])
                await page.close()

                duration = {}
                for course in courses:
                    course_page = course.replace(" ", "-").lower()
                    tmp_url = f"https://app.datacamp.com/learn/courses/{course_page}"
                    page = await context.new_page()
                    try:
                        await page.goto(tmp_url, timeout=60000)
                        await asyncio.sleep(random.uniform(2, 3))

                        soup = BeautifulSoup(await page.content(), "html.parser")
                        duration_element = soup.select_one(".mfe-app-learn-hub-hdd90k")
                        duration[course] = duration_element.get_text(strip=True) if duration_element else "N/A"
                    except Exception as e:
                        print(f"⚠️ Erro ao acessar curso '{course}': {e}")
                        duration[course] = "N/A"
                    finally:
                        await page.close()

                data = [{"Trilha": track, "Curso": course, "Duração": duration[course]} for course in courses]
                print(f"✅ Cursos extraídos com sucesso da trilha: {track}")
                return data
            else:
                print("❌ Cookies expirados ou inválidos. Refaça o login.")
                return None
        except Exception as e:
            print(f"❌ Erro ao extrair curso da página {url}: {e}")
            return None

# Função principal
async def main(url_da_trilha):
    async with async_playwright() as p:
        browser, context = await create_authenticated_context(p)
        if context is None:
            return

        data = await extract_courses(context, url_da_trilha)

        await browser.close()

        if data:
            df = pd.DataFrame(data)
            df['ordem_para_assistir'] = range(len(df)) #ordem de acordo com o html da página
            df['tipo_trilha'] = 0 #0->Trilha do datacamp, 1->trilha personalizada
            df['data_final_para_assistir'] = '' #cursos do datacamp não tem datas especificas, isso só se aplica para trilhas personalizadas
            df['obrigatoriedade_curso'] = 1 #cursos do datacamp por default sao obrigatorios, opcionalidade só se aplica para trilhas personalizadas
            print(df)
            return df
        else:
            print("Nenhum dado extraído.")
            return None


# Loop principal para pegar todas as trilhas da lista
async def rodar_todas_as_trilhas():
    for trilha_do_datacamp in LISTA_DE_TRILHAS_PARA_ATUALIZAR:
        df = await main(trilha_do_datacamp)
        if df is not None:
            df.to_excel((f'trilhas_atualizadas/{df.iloc[0]["Trilha"]}.xlsx'), index=False) #Salva o arquivo com titulo da trilha

#Executa a função
await rodar_todas_as_trilhas()

✅ Cursos extraídos com sucesso da trilha: Machine Learning Scientist in Python
                                  Trilha  \
0   Machine Learning Scientist in Python   
1   Machine Learning Scientist in Python   
2   Machine Learning Scientist in Python   
3   Machine Learning Scientist in Python   
4   Machine Learning Scientist in Python   
5   Machine Learning Scientist in Python   
6   Machine Learning Scientist in Python   
7   Machine Learning Scientist in Python   
8   Machine Learning Scientist in Python   
9   Machine Learning Scientist in Python   
10  Machine Learning Scientist in Python   
11  Machine Learning Scientist in Python   
12  Machine Learning Scientist in Python   
13  Machine Learning Scientist in Python   
14  Machine Learning Scientist in Python   
15  Machine Learning Scientist in Python   
16  Machine Learning Scientist in Python   
17  Machine Learning Scientist in Python   
18  Machine Learning Scientist in Python   
19  Machine Learning Scientist in Python 